우리가 지금까지 살펴본 예제는 스칼라 함수이거나 작은 튜퓰로 만든 상대적으로 단순한 구조를 만들어 내느 함수였다. 때로 파이썬의 변경 불가능한 이름 있는 튜플인 namedtuple을 사용해 복잡한 데이터 구조를 만들 수도 있다. 이떻게 이름 있는 튜플을 사용하고, 어떻게 그것을 만들 수 있는지에 대해 배울 것이다. 또한 상태가 있는 객체 클래스 대신 변경 불가능한 이름 있는 튜플을 사용하는 방법을 살펴본다.

객체지향 프로그래밍의 이점 중 하나는 복잡한 데이터를 점진적으로 만들어 낼 수 있는 능력이다. 어떤 측면에서 객체는 함수의 결과를 캐시해둔 것이라 할 수 있다. 그래서 객체가 함수형 디자인 패턴에도 잘 들어맞는 경우가 자주 있다. 다른 경우에 객체 패ㄷ러다임은 복잡한 계산을 포함하는 프로퍼티 메서드를 제공한다. 이 또한 함수형 설계 아이디어와 잘 들어맞는다.

하지만 어떤 경우에는 객체 클래스 정의가 복잡한 객체를 만들기 위해 상태를 활용하는 경우가 있다. 비슷한 기능을 상태가 있는 객체를 활용하지 않고 제공하는 대안에 대해 살펴본다. 상태가 있는 클래스 정의를 식별하여, 적정하거나 바른 순서로 메서드 함수 호출을 수행할 수 있도록 메타 프로퍼티를 포함시킬 것이다.

다형성 클래스 정의 바깥에서 제네릭 함수를 작성할 수 있는 몇가지 기법도 살펴본다. Callable 클래스를 활용해 다형성 클래스 계층 구조를 만들 수 있다. 이러한 구조는 함수형 설계에서 불필요한 부가 비용일 수도 있다.

### 변경 불가능한 이름 있는 튜플을 레코드로 사용하기

상황에 따라 다음 세 가지 방식 중 어느 것이든 활용할 수 있다.

* 인덱스를 사용해 이름이 붙은 원소를 선택하는 람다
* 매개변수를 사용해 인덱스에 매핑되는 매개변수 이름에 따라 원소를 선택하는 람다
* 애트리뷰트 이름이나 인덱스를 사용해 원소를 선택하는 이름 있는 튜플

튜플에서 값을 선택할 때 사용할 수 있는 세 가지 방법을 살펴보자. 첫 번째 기법은 튜플에서 인덱스를 가지고 원소를 선택하는 간단한 함수를 정의하는 것이다.

In [1]:
start = lambda leg: leg[0]
end = lambda leg: leg[1]
distance = lambda leg: leg[2]
latitude = lambda pt: pt[0]
longitude = lambda pt: pt[1]

이러한 정의가 있다면 latitude(start(first_leg))를 사용해 원하는 데이터를 참조할 수 있다.

이러한 정의는 대상 데이터 타입에 대한 정보를 그리 많이 제공하지 않는다. 이를 좀 더 분명히 하기 위해 간단한 명명 규칙을 사용할 수있다. 다음은 이름에 접두사를 붙인 선택 함수를 보여준다.

In [2]:
start_point = lambda leg: leg[0]
distance_nm = lambda leg: leg[2]
latitude_value = lambda point: point[0]

이를 주의깊게 활용하면, 이러한 명명법이 도움이 될 것이다. 이러한 방식을 너무 추구하면 각 변수  이름마다 접두사를 붙이는 헝가리 표기법처럼 복잡해질 수도 있다. 

두 번째 기법은 *매개변수 표기를 사용해 인덱스에 대한 세부 정보를 드러내는 것이다. 다음은 * 표기를 사용하는 선택 함수를 보여준다.

In [4]:
start = lambda start, end, distance: start
end = lambda start, end, distance: end
distance = lambda start, end, distance: distance
latitude = lambda lat, lon: lat
longitude = lambda lat, lon: lon

이러한 정의가 있다면, latitude(*start(*first_leg))를 사용해 데이터에서 원하는 부분을 참조할 수 있다. 이 코드는 좀 더 명확하다는 장점이 있다. 하지만 함수에 튜플을 제공하면서 *를 앞에 붙여야만 한다는 사실이 조금 이상해 보일 수 있다.

세 번째 기법은 이름 있는 튜플을 만드는 namdetuple 함수를 사용하는 것이다. 이 경우, 다음과 같이 이름 있는 튜플을 내포시켜 사용할 수 있다.

In [8]:
from collections import namedtuple
Leg = namedtuple("Leg", ("start", "end", "distance"))

이렇게 하면 first_leg.start.latitude를 사용해 데이터의 특정 부분을 가져올 수 있다. 전위 형식의 함수가 후위 형식의 애트리뷰트로 바뀐 것이 이름 있는 튜플로 쓰였다는 것을 알아챌 수 있도록 해준다. 반면, 구분이 바뀌는 것이 혼동을 초래할 수도 있다.

또한 원데이터를 처리하는 과정에서 tuple() 함수를 적당한 Leg()나 Point() 함수 호출로 변경할 것이다. 암시적으로 튜플을 만들어 내는 return이나 yield문도 찾아 바꿔줘야 한다.

경우에 따라 namedtuple 함수를 사용하면 코드가 더 명확해지기도 한다. 반면, 전위 함수에서 후위 프로퍼티로 문법만 바뀌고, 다른 실익이 없는 경우도 있다. 

### 함수형 생성자로 이름 있는 튜플 만들기

namedtuple의 인스턴스를 만드는 방법 세 가지다. 어던 기법을 선택할 것인지는 객체를 생성할 때 얼마나 많은 정보가 있느냐에 따라 달라진다.

세 기법 중 두 가지를 앞 절에서 살펴봤다. 여기서는 설계 시 고려할 점을 강조할 것이다. 설계 시에는 다음과 같은 선택 사항이 있다.

* 매개변수 값을 위치에 따라 제공할 수 있다. 평가할 식이 하나 이상 있다면 이러한 방식이 잘 작동할 것이다. Leg 객체를 만들면서 start, end에 haversiine() 함수를 적용할 때 이러한 방법을 사용했다.


* 매개변수 구문을 사용해 튜플 안에서의 위치에 따라 매개변수를 대입한다. 인자를 기존 튜플이나 반복 가능 객체로부터 기정할 경우, 이러한 방식이 잘 작동한다. map()을 사용해 float() 함수를 latitude와 longitude 값에 적용할 때 이를 사용했다.


* 키워드 대입을 활용할 수 있다. 

### 상태가 있는 클래스 사용을 피하기 위해 튜플 사용하기

지금까지 살펴본 예제에서 감싸기-풀기 디자인 패턴을 사용해 변경 불가능한 튜플이나 이름 있는 튜플을 사용할 수 있다는 것을 살펴봤다. 이러한 류의 설계에 있어 중요한 점은 변경 가능한 인스턴스 변수 대신 변경 불가능한 객체를 사용해 다른 변경 불가능한 객체를 감싸야 한다는 것이다.

### 통계적인 순위 할당하기

순위를 결정하는 문제를 두 부분으로 나눌 것이다. 첫째, Pair 객체의 x나 y값 중 하나에 순위를 부여하기 위한 제네릭한 고차 함수를 살펴본다. 그런 다음 그 함수를 사용해 Pair를 x와 y의 순위로 감싸는 함수를 만들 것이다. 이렇게 하면 내포 깊이기 깊어지는 것을 막을 수 있다.

다음은 데이터 집합의 각 관찰 결과에 순위를 부여하는 함수다.

In [2]:
from collections import defaultdict
def rank(data, key=lambda obj:obj):
    def rank_output(duplicates, key_iter, base=0):
        for k in key_iter:
            dups = len(duplicates[k])
            for value in duplicates[k]:
                yield (base+1+base+dups)/2, value
            base += dups
    def build_duplicates(duplicates, data_iter, key):
        for item in data_iter:
            duplicates[key(item)].append(item)
        return duplicates
    duplicates = build_duplicates(defaultdict(list), iter(data), key)
    return rank_output(duplicates, iter(sorted(duplicates)), 0)

순위를 만드는 함수는 Counter와 비슷한 객체를 만들어 중복 값을 찾는다. Counter 함수는 컬렉션을 만들기 위해 객체 전체를 사용하기 때문에 이 경우에 사용할 수는 없다. 각 함수에 대해 키 함수만을 사용하기 바란다. 그렇게해야만 Pair 객체의 x나 y값 중 하나를 선택해 순위를 매길 수 있다.

이 예제의 duplicates 컬렉션은 상태가 있는 객체다. 따라서 완전한 재귀함수를 만들어야 한다. 이와 같은 경우에는 데이터 집합을 나타내는 컬렉션이 큰 경우에 대비해 꼬리재귀호출 최적화를 해야만 한다. 여기에서 보여주는 것은 재귀를 최적화한 결과다.

이 재귀가 어떻게 생겼는지 이해하는 데 도움을 주기 위해 build_duplicates()에 인자를 넘겨 상태를 인자 값으로 노출하게 만들었다. 이 재귀의 기본은 data_iter가 비어 있는 경우다. data_iter에 원소가 있다면, 새로운 컬렉션을 오래된 컬렉션과 next(data_iter)로부터 만든다. build_duplicates()를 재귀적으로 평가하면 data_iter의 꼬리 부분에 있는 모든 원소를 처리할 것이다.

이와 마찬가지로 순위를 할당한 컬렉션을 내놓는 재귀적인 함수를 두 가지 작성해야 한다. 여기서도 재귀를 내포된 for 루프로 최적화한 결과를 보였다. 순위를 어떻게 계산하는지 분명히 알기 위해 범위의 최솟값과 범위의 최댓값을 포함시키고, 이 두 값의 중간 값을 취했다. 중복이 하나밖에 없다면, (2*base+2) / 2를 계산한다. 따라서 이 계산이 일반적인 해법이 될 수 있다는 이점이 있다.

다음은 이 함수가 제대로 작동하는지 어떻게 테스트할 수 있는지 보여준다.

In [3]:
list(rank([0.8, 1.2, 1.2, 2.3, 18]))

[(1.0, 0.8), (2.5, 1.2), (2.5, 1.2), (4.0, 2.3), (5.0, 18)]

In [4]:
data = ((2, 0.8), (3, 1.2), (5, 1.2), (7, 2.3), (11, 18))

In [5]:
list(rank(data, key=lambda x:x[1]))

[(1.0, (2, 0.8)),
 (2.5, (3, 1.2)),
 (2.5, (5, 1.2)),
 (4.0, (7, 2.3)),
 (5.0, (11, 18))]

표본 데이터에는 같은 값이 2개 들어 있다. 결과에서 두 번째와 세 번쨰 위치에 있는 표본에 2.5라는 순위를 부여한다. 이러한 식으로 순위를 부여하는 것은 두 집합사이의 스피어맨 순위 상관계수를 계산할 때 일반적으로 사용되는 통계적인 방식이다.

### 상태를 바꾸는 대신 감싸기

어떤 데이터를 감싸는 데에는 두 가지 일반적인 전략이 있다.

* 병렬성: 데이터의 복사본을 두 가지 만들어 각각의 순위를 계산한다. 그 후 두 복사본을 양쪽 결과를 모두 포함하는 최종 결과로 다시 합친다. 순서가 다를 수도 있는 두 시퀀스를 병합해야 하기 때문에 이렇게 하는 것이 약간 이상할 수 있다.

* 직렬성: 한 변수에 대한 순위를 계산한 결과로 원래의 데이터를 감싼다. 그 후 이렇게 감싼 데이터를 다른 병수에 대한 순위로 부여할 수 있다. 이렇게 하면 좀 더 복잡한 구조가 생긴다. 하지만 마지막 결과를 감쌀 때 정보를 평면적으로 펼치도록 하는 방식으로 어느 정도 최적화할 수 있다.

다음은 Pair를 y 값의 순위로 감싸는 객체를 어떻게 만드는지 보여준다.

In [8]:
from collections import namedtuple

Rank_Y = namedtuple("Rank_Y", ("r_y", "raw",))
def rank_y(pairs):
    return (Rank_Y(*row)
    for row in rank(pairs, lambda pair: pair.y))

y값의 순위와 원래의 값을 포함하는 namedtuple 함수를 정의했다. rank_y() 함수는 Pair 객체에서 y를 선택하는 람다를 rank() 함수에 적용하여 이러한 이름있는 튜플을 만든다. 그 결과 2-튜플의 인스턴스를 만든다.

여기서 알아둘 것은 다음과 같은 입력을 제공하는 경우,

In [10]:
data = (Pair(x=10.0, y=8.04), Pair(x=8.0, y=6.95), Pair(x=5.0, y=5.68))

원래의 Pair 객체의 순위를 포함하는 새로운 객체가 감싼다. 이것이 필요로 하는 전부는 아니다. 이 데이터를 한 번 더 감싸 x와 y의 순위 정보를 모두 포함하는 객체를 만들어야 한다.

### 상태를 바꾸는 대신 다시 감싸기
r_x와 ranked_y라는 애트리뷰트를 가진 Ranked_X라는 이름의 namedtuple을 사용할 수 있다. 

### 스피어맨 순위 상관계수 계산하기 

스피어맨 순위 상관계수는 두 변수의 순위를 비교하는 것이다. 이 상관계수는 각 값의 규모를 깔끔하게 우회할 수 있고, 종종 두 변수 간의 관계가 선형이 아닌 경우의 상관관계를 찾아낼 수도 있다. 

공식을 보면 모든 관찰 값의 쌍 사이에 x_i와 y_i의 순위 차이에 대한 합계를 계산해야 한다는 것을 보여준다. 파이썬으로 sum()과 len() 함수를 사용해 이를 작성하면 다음과 같다.

In [11]:
def rank_corr(pairs):
    ranked = rank_xy(pairs)
    sum_d_2 = sum((r.r_x - r.r_y)**2 for r in ranked)
    n = len(pairs)
    return 1-6*sum_d_2/(n*(n**2-1))

각 쌍에 대해 Ranked_XY 객체를 만들었다. 이렇게 만든 각 값에서 r_x와 r_y 값을 빼서 차이를 구한다. 그런 다음, 그 차이의 합계와 제곱을 계산한다.

통계에 대한 좋은 글이라면 이 상관계수의 의미를 잘 설명해줄 것이다. 이 계수가 0 근처에 있다면 두 데이터 계열 사이에 상관관계가 없다는 의미다. 산점도를 보면 점이 임의로 분포한 것을 볼 수 있다. 값이 +1이나 -1 근처에 있다면 두 변수 사이에 강력한 상관관계가 있다는 것을 의미한다. 그래프를 보면 깔끔한 선이나 곡선을 볼 수 있다.

### 다형성과 파이썬다운 패턴 매치

일부 함수형 언어네는 정적으로 탕비을 지정하는 함수 정의에 대해 작업할 수 있는 멋진 접근 방법을 제공한다. 문제가 되는 것은 작성하려는 많은 함수가 데이터 타입을 기준으로 볼 때 완전히 제네릭한 경우다.

파이썬에서는 코드가 특정 데이터 타입에 한정되지 않기 때문에 사실상 제네릭 정의를 작성하는 것이나 마찬가지다. 파이썬 런타임이 단순한 규칙을 사용해 적절한 연산자를 선택해준다. 